# Quantized Neural Network
## 05- Training
by [Soon Yau Cheong](http://www.linkedin.com/in/soonyau)

Finally, we will put everything we have learned so far and code them up for training. In this tutorial, we'll first train a MobileNet on Cifar10 using full precision, then we do quantization-aware training ranging from 8 to 2 bits to compare the accuracy loss.

### Setup
The setup of this tutorial:
- Network Architecture : Mobilenet v1
- Dataset: Cifar10
- Framework : Tensorflow Estimator
- Visualization: Tensorboard

With boilerplate code, I think the code becomes a bit too long to put in a single Jupyter notebook. Therefore, I split them into multiple files and put into directory /training where you can run the training by doing
```
python train.py
```

Files structures:
- *generate_cifar10_tfrecords.py*: run this file to download Cifar10 and convert that into tfrecord for tf.data pipeline
- *cifar10.py*: return pre-processed batches of Cifar10 images and labels
- *config.py*: training configuration e.g. learning rate, directory
- *train.py*: the main code using tf.Estimator to pull in the data pipepine and QNN model for training and evaluation
- *mobilenet.py*: our model including all the quantization code


## Quantized Layers

Although we can use tf.contrib.quantize.create_training_graph() to add fake quantization to a graph but at the time of writing, that API doesn't allow us to change the bitwidth. Therefore, it is always 8 bits so if we want to run experimentation, then we'll need to write our own quantized layers. We have already learned how to do that by using lower level APIs to create variable and pass that through tf.fake_quant_with_min_max_vars(). So now we will put them together into a function \_conv() like below. The argument *num_bits* is used to set the bitwidth, if it is *None*, then no quantization is performed.

In [1]:
    def _conv2d(self, 
            num_bits,
            x,             
            n_output_plane,
            kernel_size,
            strides=1,
            bias=False,
            padding='SAME',
            name='conv2d'):
        with tf.variable_scope(name):
            n_input_plane = x.get_shape().as_list()[3]
            w_dim = [kernel_size, kernel_size, n_input_plane, n_output_plane]
            w = tf.get_variable("weight", w_dim, 
                initializer=tf.contrib.layers.xavier_initializer_conv2d())
            if num_bits:
                w_min = tf.reduce_min(w)
                w_max = tf.reduce_max(w)
                w = tf.fake_quant_with_min_max_vars(w, w_min, w_max, num_bits)

            output = tf.nn.conv2d(x, w, [1, strides, strides,1], padding)

            if bias:
                b = tf.get_variable('bias', [n_output_plane])
                output =  tf.nn.bias_add(output, b)

        return output

Same goes for *\_relu6()* and *\_depthwise_conv2d()*. It would be quite tedious to set num_bits for each of the layers. Good news is we could hide them using some good software engineering practices and that's exactly what we will do. 

We create a MobileNet class and pass num_bits in the constructor and use that to create partial functions of layers. With that, we can just call self.conv2d(x,...) instead of self.\_conv2d(num_bits=8, x,...) and we can change the quantization bidwidth easily by passing different value when instantiating MobileNet class.

In [2]:
class MobileNet():
    def __init__(self, num_class, is_training, num_bits=None, width_multiplier=1):
        self.relu6 = partial(self._relu6, num_bits)
        self.conv2d = partial(self._conv2d, num_bits)
        self.depthwise_conv2d = partial(self._depthwise_conv2d, num_bits)

### Seperable Conv2d

Apart from the first and last layers, MobileNet is wholly made up of seperable conv2d layers, therefore we create one that encapsulate "depthwise_conv2d-bn-relu-conv2d_1x1-bn-relu" operations.

In [ ]:
    def separable_conv2d(self, 
            x,             
            kernel_size,
            filters,
            strides=1,
            bias=False,
            padding='SAME',
            name='separable_conv2d'):
        with tf.variable_scope(name):
            filter_depth = self.width_multiplier*np.array(filters)
            x = self.depthwise_conv2d(x, 
                    int(filter_depth[0]), 
                    kernel_size, 
                    strides, 
                    bias, 
                    padding)

            x = tf.layers.batch_normalization(x, training=self.is_training)
            x = self.relu6(x)

            x = self.conv2d(x, 
                    int(filter_depth[1]), 
                    kernel_size=1, 
                    strides=1, 
                    bias=bias, 
                    padding=padding)
            x = tf.layers.batch_normalization(x, training=self.is_training)
            x = self.relu6(x)
        return x


### MobileNet

With all the layers defined, we can now define the MobiletNet architecture. Note that we transformed the fully connected layer (filter shape 1024x1000) into a 1x1 conv2d with 1000 filters, the underlying operations are identical and we could now avoid reshaping before the fully connected layer. 

In [5]:
    def forward_pass(self, x):

        filter_depth = np.int(self.width_multiplier*32)
        x = self.conv2d(x, filter_depth, 3, 2)
        x = tf.layers.batch_normalization(x, training=self.is_training)
        x = self.relu6(x)

        x = self.separable_conv2d(x, 3, [32, 64], 1, name='separable_1')
        x = self.separable_conv2d(x, 3, [64, 128], 2, name='separable_2')
        x = self.separable_conv2d(x, 3, [128, 128], 1, name='separable_3')
        x = self.separable_conv2d(x, 3, [128, 256], 2, name='separable_4')
        x = self.separable_conv2d(x, 3, [256, 256], 1, name='separable_5')
        x = self.separable_conv2d(x, 3, [256, 512], 2, name='separable_6')
        x = self.separable_conv2d(x, 3, [512, 512], 1, name='separable_7')
        x = self.separable_conv2d(x, 3, [512, 512], 1, name='separable_8')
        x = self.separable_conv2d(x, 3, [512, 512], 1, name='separable_9')
        x = self.separable_conv2d(x, 3, [512, 512], 1, name='separable_10')
        x = self.separable_conv2d(x, 3, [512, 512], 1, name='separable_11')
        x = self.separable_conv2d(x, 3, [512, 1024], 2, name='separable_12')
        x = self.separable_conv2d(x, 3, [1024, 1024], 1, name='separable_13')

        pool_size = x.get_shape().as_list()[1]
        x = tf.layers.average_pooling2d(x, pool_size=pool_size, strides=1)
        
        x = tf.layers.dropout(x, training=self.is_training)
        
        x = self.conv2d(x, self.num_class, 1,1, name='fc')
        x = tf.squeeze(x, [1, 2], 'spatial_squeeze')

        x_shape = x.get_shape().as_list()
        tf.logging.info("output tensor: %s", x.get_shape())
        return x

## Training

In MobileNet, there is a parameter width_multiplier with range [0, 1] to thin the number of filters. As Cifar10 has image dimension of only 32x32, we could reduce the network size by setting width multiplier to 0.75 or even 0.5 to speed up training and reduce overfitting.

We use Adam Optimizer and the hyperparameters as defined in config.py. 

```
width_multiplier=0.75
train_batch_size=256
eval_batch_size=100
learning_rate=0.01
decay_rate=0.94
decay_per_epoch=2
num_epoch=100
```

To run the training, go to directory /training and type:
```
python train.py
```

To look at the accuracy and loss graph
```
tensorboard --logdir=log/custom
```
where it will collect data in all subdirectories and plot them into a single graph. Or you can look at individual quantization graph
```
tensorboard --logdir=log/custom/numbits_8
```

I designate the name "numbits_9" to mean no quantization, so the labels arrange in order of quantization bitwidth like the result shown below.



## Results

When I first trained the model, I got top accuracy of 0.81 then I added weight normalization and decay to reduce overfitting then I got 0.85 top-1 accuracy.

If we look at the accuracy on evaluation set, we can see that quantization to 8 bits bear no loss in accuracy compared to full precision, as claimed by Google in their work on Imagenet! Of course, Cifar10 is much smaller than Imagenet in both image resolution and sample sizes but we can see that quantization can work pretty well. We could use smaller width multiplier to further reduce MobileNet size and re-run the experiments. If we look closer to the numbers, 5-bits's accuracy is pretty close to 8-bits. We can also see the accuracy drops significantly for quantization of 4 bits and below. However, this is an active research area and accuracy may be boosted by using different quantization and training techniques. 

In the future, quantization bitwidth may well be a new hyperparameter to tune.


![](images/accuracy_graph2.png)

## What's Next?

Today we learned how to perform quantization-aware training. Next, we'll look at how a trained graph is transformed, optimized and quantized into integers.
